In [20]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [21]:
df = pd.read_csv("transfusion_10%.csv")

In [22]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,NaN,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,NaN,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,NaN,0.0
744,NaN,2.0,NaN,52.0,0.0
745,NaN,NaN,NaN,62.0,0.0
746,39.0,1.0,250.0,NaN,0.0


In [23]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [24]:
df.isnull().sum()

Recency (months)                              107
Frequency (times)                              78
Monetary (c.c. blood)                          94
Time (months)                                  95
whether he/she donated blood in March 2007      0
dtype: int64

In [25]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 374
persentase : 10.0


In [26]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              14.304813
Frequency (times)                             10.427807
Monetary (c.c. blood)                         12.566845
Time (months)                                 12.700535
whether he/she donated blood in March 2007     0.000000
dtype: float64

# Imputasi

In [27]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [28]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,3)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [29]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [30]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.0,50.0,0.0,98.0
1,0.0,13.0,3250.0,28.0
2,1.0,16.0,4000.0,0.0
3,2.0,20.0,5000.0,45.0
4,1.0,24.0,6000.0,77.0
...,...,...,...,...
743,23.0,2.0,500.0,0.0
744,0.0,2.0,0.0,52.0
745,0.0,0.0,0.0,62.0
746,39.0,1.0,250.0,0.0


In [31]:
euc_dist = {}

In [32]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,3)

Kolom : 1
[2.0, 4.0, 1.0]
mean :  2.3333333333333335
standar deviasi :  1.247219128924647
fuzzy :  [(2.0, 0.9649195174720683), (4.0, 0.40952203499907786), (1.0, 0.5647515815057149)]
fuzzy :  None
pembilang :  4.132678756446163
penyebut :  1.939193133976861
rata-rata tertimbang :  2.131133141942877
 
[4.0, 0.0, 4.0]
mean :  2.6666666666666665
standar deviasi :  1.8856180831641267
fuzzy :  [(4.0, 0.778820970680917), (0.0, 0.36791758646063283), (4.0, 0.778820970680917)]
fuzzy :  None
pembilang :  6.230567765447336
penyebut :  1.9255595278224669
rata-rata tertimbang :  3.235718073329688
 
[11.0, 1.0, 5.0]
mean :  5.666666666666667
standar deviasi :  4.109609335312651
fuzzy :  [(11.0, 0.43084023153866097), (1.0, 0.5248357129715819), (5.0, 0.9869296383686192)]
fuzzy :  None
pembilang :  10.198726451739947
penyebut :  1.9426055828788622
rata-rata tertimbang :  5.250024267214269
 
[2.0, 2.0, 0.0]
mean :  1.3333333333333333
standar deviasi :  0.9428090415820634
fuzzy :  [(2.0, 0.778820970680917

[0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  2
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 2.0, 2.0]
mean :  1.3333333333333333
standar deviasi :  0.9428090415820634
fuzzy :  [(0.0, 0.36791758646063283), (2.0, 0.778820970680917), (2.0, 0.778820970680917)]
fuzzy :  None
pembilang :  3.115283882723668
penyebut :  1.9255595278224669
rata-rata tertimbang :  1.617859036664844
 
[0.0, 0.0, 4.0]
mean :  1.3333333333333333
standar deviasi :  1.8856180831641267
fuzzy :  [(0.0, 0.778820970680917), (0.0, 0.778820970680917), (4.0, 0.3679175864606327)]
fuzzy :  None
pembilang :  1.4716703458425309
penyebut :  1.9255595278224669
rata-rata tertimbang :  0.7642819266703118
 
[0.0, 0.7642819266703118, 4.0]
mean :  1.5880939755567705
standar deviasi :  1.7337819391757199
fuzzy :  [(0.0, 0.6574030750534474), (0.7642819266703118, 0.8932635787339637), (4.0, 0.3800270089024207)]
fuzzy :  None
pembilang :  2.2

[0.0, 2.6749867433460928, 3.993303808393828]
mean :  2.2227635172466402
standar deviasi :  1.6613244136734224
fuzzy :  [(0.0, 0.4086248080767131), (2.6749867433460928, 0.9636334114812104), (3.993303808393828, 0.566747015990864)]
fuzzy :  None
pembilang :  4.840899618509763
penyebut :  1.9390052355487875
rata-rata tertimbang :  2.4965892457426326
 
[0.0, 4.0, 2.0]
mean :  2.0
standar deviasi :  1.632993161855452
fuzzy :  [(0.0, 0.47240328690554567), (4.0, 0.47240328690554567), (2.0, 1.0)]
fuzzy :  None
pembilang :  3.8896131476221827
penyebut :  1.9448065738110913
rata-rata tertimbang :  2.0
 
[0.0, 4.0, 16.0]
mean :  6.666666666666667
standar deviasi :  6.79869268479038
fuzzy :  [(0.0, 0.6183384098997249), (4.0, 0.9259684638717255), (16.0, 0.3897655050399652)]
fuzzy :  None
pembilang :  9.940121936126346
penyebut :  1.9340723788114156
rata-rata tertimbang :  5.139477738798508
 
[0.0, 11.0, 0.0]
mean :  3.6666666666666665
standar deviasi :  5.185449728701348
fuzzy :  [(0.0, 0.7788209706

[0.0, 1.617859036664844, 2.0]
mean :  1.2059530122216147
standar deviasi :  0.8668909695878599
fuzzy :  [(0.0, 0.38002700890242086), (1.617859036664844, 0.8932635787339637), (2.0, 0.6574030750534473)]
fuzzy :  None
pembilang :  2.7599807030852164
penyebut :  1.9306936626898317
rata-rata tertimbang :  1.4295280273723106
 
[0.0, 0.0, 0.7642819266703118]
mean :  0.25476064222343725
standar deviasi :  0.3602859553912648
fuzzy :  [(0.0, 0.778820970680917), (0.0, 0.778820970680917), (0.7642819266703118, 0.3679175864606327)]
fuzzy :  None
pembilang :  0.2811927618360234
penyebut :  1.9255595278224669
rata-rata tertimbang :  0.14603171585872096
 
[0.0, 2.0, 13.0]
mean :  5.0
standar deviasi :  5.715476066494082
fuzzy :  [(0.0, 0.6820765485778446), (2.0, 0.8713244932332572), (13.0, 0.37550245551847183)]
fuzzy :  None
pembilang :  6.6241809082066485
penyebut :  1.9289034973295736
rata-rata tertimbang :  3.434169162623918
 
[0.0, 0.7642819266703118, 1.140943945255379]
mean :  0.6350752906418969
s

[7.0, 7.0, 7.0]
mean :  7.0
standar deviasi :  0.0
median :  2
fuzzy :  [(7.0, 7.0), (7.0, 7.0), (7.0, 7.0)]
fuzzy :  7.0
pembilang :  147.0
penyebut :  21.0
rata-rata tertimbang :  7.0
 
[6.0, 6.0, 6.0]
mean :  6.0
standar deviasi :  0.0
median :  2
fuzzy :  [(6.0, 6.0), (6.0, 6.0), (6.0, 6.0)]
fuzzy :  6.0
pembilang :  108.0
penyebut :  18.0
rata-rata tertimbang :  6.0
 
[2.0, 2.0, 2.0]
mean :  2.0
standar deviasi :  0.0
median :  2
fuzzy :  [(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]
fuzzy :  2.0
pembilang :  12.0
penyebut :  6.0
rata-rata tertimbang :  2.0
 
[0.0, 13.0, 13.0]
mean :  8.666666666666666
standar deviasi :  6.128258770283412
fuzzy :  [(0.0, 0.3679175864606329), (13.0, 0.778820970680917), (13.0, 0.778820970680917)]
fuzzy :  None
pembilang :  20.249345237703842
penyebut :  1.9255595278224669
rata-rata tertimbang :  10.516083738321486
 
[5.0, 5.0, 5.0]
mean :  5.0
standar deviasi :  0.0
median :  2
fuzzy :  [(5.0, 5.0), (5.0, 5.0), (5.0, 5.0)]
fuzzy :  5.0
pembilang :  75.0
peny

...

[0.0, 1250.0, 1500.0]
mean :  916.6666666666666
standar deviasi :  656.1673228343176
fuzzy :  [(0.0, 0.37692642270039506), (1250.0, 0.8789573711581657), (1500.0, 0.6735989880796183)]
fuzzy :  None
pembilang :  2109.0951960671346
penyebut :  1.929482781938179
rata-rata tertimbang :  1093.0883736358267
 
[0.0, 750.0, 500.0]
mean :  416.6666666666667
standar deviasi :  311.8047822311618
fuzzy :  [(0.0, 0.4095220349990779), (750.0, 0.5647515815057151), (500.0, 0.9649195174720684)]
fuzzy :  None
pembilang :  906.0234448653205
penyebut :  1.9391931339768615
rata-rata tertimbang :  467.2167145142806
 
[0.0, 0.0, 47.76762041689449]
mean :  15.922540138964829
standar deviasi :  22.51787221195405
fuzzy :  [(0.0, 0.778820970680917), (0.0, 0.778820970680917), (47.76762041689449, 0.3679175864606329)]
fuzzy :  None
pembilang :  17.57454761475147
penyebut :  1.9255595278224669
rata-rata tertimbang :  9.126982241170065
 
[0.0, 9.126982241170065, 47.76762041689449]
mean :  18.964867552688187
standar de

[2.6749867433460928, 3.993303808393828, 14.0]
mean :  6.889430183913308
standar deviasi :  5.056655172679623
fuzzy :  [(2.6749867433460928, 0.7066080259577017), (3.993303808393828, 0.8487453985876977), (14.0, 0.3721086987286475)]
fuzzy :  None
pembilang :  10.48898711691686
penyebut :  1.927462123274047
rata-rata tertimbang :  5.4418641955463904
 
[0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  2
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[71.0, 0.0, 26.74352278970405]
mean :  32.58117426323468
standar deviasi :  29.27807633759216
fuzzy :  [(71.0, 0.4228020576763809), (0.0, 0.5384191678894088), (26.74352278970405, 0.9803207983069561)]
fuzzy :  None
pembilang :  56.23617770576599
penyebut :  1.9415420238727457
rata-rata tertimbang :  28.964697654904775
 
[0.0, 0.0, 4.0]
mean :  1.3333333333333333
standar deviasi :  1.8856180831641267
fuzzy :  [(0.0, 0.778820970680917), (0.0, 0.778820970680917), (

[0.0, 33.0, 40.0]
mean :  24.333333333333332
standar deviasi :  17.441967269268172
fuzzy :  [(0.0, 0.3779268248737231), (33.0, 0.8838791158208728), (40.0, 0.6680750148995619)]
fuzzy :  None
pembilang :  55.89101141807127
penyebut :  1.9298809555941578
rata-rata tertimbang :  28.960859609531692
 
[0.0, 9.0, 9.0]
mean :  6.0
standar deviasi :  4.242640687119285
fuzzy :  [(0.0, 0.3679175864606327), (9.0, 0.778820970680917), (9.0, 0.778820970680917)]
fuzzy :  None
pembilang :  14.018777472256506
penyebut :  1.9255595278224669
rata-rata tertimbang :  7.280365664991797
 
[0.0, 0.0, 14.0]
mean :  4.666666666666667
standar deviasi :  6.599663291074443
fuzzy :  [(0.0, 0.778820970680917), (0.0, 0.778820970680917), (14.0, 0.3679175864606329)]
fuzzy :  None
pembilang :  5.150846210448861
penyebut :  1.9255595278224669
rata-rata tertimbang :  2.6749867433460928
 
[0.0, 64.0, 87.0]
mean :  50.333333333333336
standar deviasi :  36.80881536926839
fuzzy :  [(0.0, 0.3926525809369381), (64.0, 0.933401167

[0.0, 28.0, 35.0]
mean :  21.0
standar deviasi :  15.121728296285006
fuzzy :  [(0.0, 0.3812935481132321), (28.0, 0.8984073017254401), (35.0, 0.6514680055958544)]
fuzzy :  None
pembilang :  47.95678464416723
penyebut :  1.9311688554345265
rata-rata tertimbang :  24.83303544856341
 
[0.0, 0.7642819266703118, 1.140943945255379]
mean :  0.6350752906418969
standar deviasi :  0.4746641181924061
fuzzy :  [(0.0, 0.408624808076713), (0.7642819266703118, 0.9636334114812104), (1.140943945255379, 0.5667470159908639)]
fuzzy :  None
pembilang :  1.3831141767170743
penyebut :  1.9390052355487872
rata-rata tertimbang :  0.7133112130693232
 
[0.0, 13.371761394852024, 14.0]
mean :  9.123920464950674
standar deviasi :  6.456682038751734
fuzzy :  [(0.0, 0.36849852120547766), (13.371761394852024, 0.8054189287683311), (14.0, 0.7519141192338029)]
fuzzy :  None
pembilang :  21.296667407660685
penyebut :  1.9258315692076118
rata-rata tertimbang :  11.058426784655552
 
[0.0, 0.14603171585872096, 0.7642819266703

[0.0, 38.0, 38.0]
mean :  25.333333333333332
standar deviasi :  17.9133717900592
fuzzy :  [(0.0, 0.3679175864606327), (38.0, 0.7788209706809169), (38.0, 0.7788209706809169)]
fuzzy :  None
pembilang :  59.19039377174969
penyebut :  1.9255595278224664
rata-rata tertimbang :  30.73932169663204
 


In [33]:
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.000000,50.000000,9090.834643,98.000000
1,2.131133,13.000000,3250.000000,28.000000
2,1.000000,16.000000,4000.000000,56.993522
3,2.000000,20.000000,5000.000000,45.000000
4,1.000000,24.000000,6000.000000,77.000000
...,...,...,...,...
743,23.000000,2.000000,500.000000,18.605379
744,1.140944,2.000000,101.506773,52.000000
745,4.384002,8.089295,2113.190467,62.000000
746,39.000000,1.000000,250.000000,30.739322


In [34]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
count,748.000000,748.000000,748.000000,748.000000
mean,8.803392,5.436185,1336.838704,32.882425
std,7.852588,5.839027,1415.526854,24.821951
min,0.146032,0.191070,8.518295,0.073016
25%,2.000000,2.000000,500.000000,14.000000
50%,4.713662,4.000000,1000.000000,28.000000
75%,14.000000,7.000000,1750.000000,48.692156
max,74.000000,50.000000,11500.000000,98.000000


# Klasifikasi

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Gantilah df2 dengan data Anda sendiri (x dan y)
x = df2.iloc[:, :-1].values
y = df2.iloc[:, -1].values

# Bagi data menjadi set pelatihan dan pengujian
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=15)

# Normalisasi data menggunakan MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Membangun model
model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Kompilasi model dengan binary cross-entropy sebagai fungsi kerugian
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Menambahkan Early Stopping untuk menghindari overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Melatih model
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluasi model pada data pengujian
accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy[1]}')

Epoch 1/100
16/16 [==============================] - 2s 16ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/100
16/16 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/100
6/6 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.0000e+00
Test accuracy: 0.0


In [36]:
x = dummy.values
y = df.iloc[:, -1].values

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=72)

In [38]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)